In [1]:
#Dependencies
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import tensorflow as tf

In [2]:
#read csv into df
df = pd.read_csv("Levels_Fyi_Salary_Data 2.csv")
# columns to drop... salary_currency, salary, work_year
df.sample(25)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
24487,7/19/2020 15:02:39,Amazon,L4,Software Engineer,150000,"Irvine, CA",0.0,0.0,Distributed Systems (Back-End),112000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
38409,12/31/2020 17:55:52,Bloomberg,Software Engineer,Software Engineer,175000,"New York, NY",0.0,0.0,Distributed Systems (Back-End),155000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
48211,4/14/2021 4:07:20,Oracle,IC-2,Data Scientist,205000,"Redwood City, CA",1.0,0.0,NLP,140000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
26408,8/13/2020 0:10:35,Tesla,P3,Hardware Engineer,220000,"Palo Alto, CA",5.0,3.0,Power Electronics,150000.0,...,1,0,0,0,0,0,0,0,NaN,PhD
45919,3/24/2021 10:41:50,Change.org,L3,Software Engineer,133000,"San Francisco, CA",3.0,3.0,Full Stack,133000.0,...,0,0,0,0,0,0,1,0,Black,Bachelor's Degree
44106,3/6/2021 15:42:54,USAA,3,Software Engineer,73000,"Plano, TX",2.0,2.0,Full Stack,66000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
18764,3/27/2020 17:10:59,Google,L6,Technical Program Manager,400000,"Los Angeles, CA",18.0,5.0,Web Development (Front-End),200000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
20037,4/25/2020 18:00:43,Microsoft,66,Software Engineering Manager,327000,"Redmond, WA",20.0,20.0,Mobile (iOS + Android),210000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
16764,2/15/2020 6:04:45,Morgan Stanley,Manager,Software Engineering Manager,30000,"Bengaluru, KA, India",6.0,1.0,Web Development (Front-End),30000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
10081,8/26/2019 16:52:49,Genesys,Software Engineer,Software Engineer,102000,"Indianapolis, IN",12.0,6.0,API Development (Back-End),0.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [3]:
df.nunique()


timestamp                  62561
company                     1631
level                       2923
title                         15
totalyearlycompensation      893
location                    1050
yearsofexperience             65
yearsatcompany                81
tag                         3058
basesalary                   482
stockgrantvalue              612
bonus                        335
gender                         4
otherdetails               12841
cityid                      1045
dmaid                        149
rowNumber                  62642
Masters_Degree                 2
Bachelors_Degree               2
Doctorate_Degree               2
Highschool                     2
Some_College                   2
Race_Asian                     2
Race_White                     2
Race_Two_Or_More               2
Race_Black                     2
Race_Hispanic                  2
Race                           5
Education                      5
dtype: int64

In [4]:
titles_df = df[df["title"]=="Data Scientist"]
titles_df


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
419,6/5/2018 14:06:30,LinkedIn,Senior,Data Scientist,233000,"San Francisco, CA",4.0,0.0,Data Analysis,162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
440,6/8/2018 9:49:25,Microsoft,64,Data Scientist,218000,"Seattle, WA",11.0,11.0,ML / AI,165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
444,6/8/2018 17:55:09,ebay,26,Data Scientist,180000,"San Jose, CA",10.0,5.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,6/10/2018 19:39:35,Twitter,Staff,Data Scientist,500000,"San Francisco, CA",4.0,4.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
495,6/17/2018 11:39:38,Facebook,5,Data Scientist,370000,"Seattle, WA",8.0,3.0,NaN,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62240,3/26/2019 17:25:02,IBM,8,Data Scientist,155000,"Washington, DC",5.0,2.0,ML / AI,141000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62283,6/17/2018 21:07:18,Facebook,E3,Data Scientist,150000,"Singapore, SG, Singapore",0.0,0.0,ML / AI,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62285,7/16/2018 20:11:40,ADP,Sr UX Researcher 4,Data Scientist,185000,"Roseland, NJ",8.0,2.0,UX Research,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62529,6/12/2018 20:54:06,Google,T6,Data Scientist,685000,"Kirkland, WA",22.0,2.0,ML / AI,221000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [5]:
#Binning years of experience for better encoding
titles_df['yearsofexperience'].value_counts()



2.0     351
5.0     343
3.0     314
4.0     301
1.0     214
0.0     190
6.0     178
7.0     141
10.0    132
8.0     124
9.0      64
15.0     45
12.0     42
11.0     37
13.0     25
20.0     21
16.0     18
14.0     15
17.0      6
18.0      4
25.0      4
23.0      2
22.0      2
19.0      1
26.0      1
30.0      1
21.0      1
34.0      1
Name: yearsofexperience, dtype: int64

In [6]:
# Replace in dataframe
years_labels = []
for years in titles_df['yearsofexperience']:
    if years < 5.0:
        years = "Entry Level"
    elif (years>=5.0 and years <10.0):
        years = "Junior"
    else:
        years = "Senior"
    years_labels.append(years)
years_labels

['Entry Level',
 'Senior',
 'Senior',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Senior',
 'Senior',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Senior',
 'Junior',
 'Junior',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Senior',
 'Entry Level',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Junior',
 'Junior',
 'Junior',
 'Junior',
 'Entry Level',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Entry Level',
 'Senior',
 'Entry Level',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Entry Level',
 'Junior',
 'Junior',
 'Junior',
 'Senior',
 'Senior',
 'Entry Level',
 'Entry Level',
 'Senior',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Entry Level',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Junior',
 'Junior',
 'Entry Level',
 'Entry Level',
 'Entry Level',
 'Entry Level',
 'Senior',
 'Entry Level',
 'Entry Level',
 'Entry Level',
 'Junior',
 'Entry Level',
 'Senior',
 'Entry Level',
 'Junior',
 'Junior',
 'Junior',
 'Junior',
 'Entry Level',
 'Entry Level',
 

In [7]:
#reassign company column to bin name
year_level_df = titles_df.copy()
year_level_df['yearsofexperience'] = years_labels
year_level_df.head(20)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
419,6/5/2018 14:06:30,LinkedIn,Senior,Data Scientist,233000,"San Francisco, CA",Entry Level,0.0,Data Analysis,162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
440,6/8/2018 9:49:25,Microsoft,64,Data Scientist,218000,"Seattle, WA",Senior,11.0,ML / AI,165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
444,6/8/2018 17:55:09,ebay,26,Data Scientist,180000,"San Jose, CA",Senior,5.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,6/10/2018 19:39:35,Twitter,Staff,Data Scientist,500000,"San Francisco, CA",Entry Level,4.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
495,6/17/2018 11:39:38,Facebook,5,Data Scientist,370000,"Seattle, WA",Junior,3.0,NaN,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
499,6/17/2018 19:02:50,Amazon,L5,Data Scientist,200000,"Seattle, WA",Entry Level,0.0,ML / AI,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
509,6/20/2018 0:47:43,Microsoft,65,Data Scientist,340000,"Bellevue, WA",Senior,11.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
510,6/20/2018 0:49:11,Google,L6,Data Scientist,690000,"Kirkland, WA",Senior,0.0,ML / AI,240000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
513,6/21/2018 10:54:35,Netflix,Senior,Data Scientist,600000,"Los Gatos, CA",Entry Level,1.0,ML / AI,600000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
523,6/25/2018 8:45:29,Tesla,Senior Engineer,Data Scientist,168000,"Palo Alto, CA",Junior,3.0,Mechanical Engineering,118000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [8]:
year_level_df['yearsofexperience'].value_counts()

Entry Level    1370
Junior          850
Senior          358
Name: yearsofexperience, dtype: int64

In [9]:
clean_df= year_level_df.drop(columns=['timestamp', 'level', 'yearsatcompany', 'otherdetails','Highschool', 'Some_College', 
                               'Race_Asian', 'Race_White', 'Race_Two_Or_More', 'Race_Black',
                              'Race_Hispanic', 'Race', 'Education','cityid','dmaid','rowNumber','tag','bonus','stockgrantvalue','basesalary','title'])
clean_df.head()
#get_dummies/oneHotEncoder on company size and employment type

,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",Entry Level,Male,0,0,0
440,Microsoft,218000,"Seattle, WA",Senior,Male,0,0,0
444,ebay,180000,"San Jose, CA",Senior,Female,0,0,0
454,Twitter,500000,"San Francisco, CA",Entry Level,Male,0,0,0
495,Facebook,370000,"Seattle, WA",Junior,Male,0,0,0


In [10]:
import re
form_state_one = r'(\w+\s)*(\w+),\s([A-Z]{2})$'
matches_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False)
no_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False).sum()
print(no_form_one)

form_state_two = r'(\w+\s)*(\w+),\s([A-Z]{2}),\s(\w+)'
matches_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False)
no_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False).sum()
print(no_form_two)


clean_df.location[~matches_form_one & ~matches_form_two]

2226
351


/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


9061    Tel Aviv, Israel
Name: location, dtype: object

In [11]:
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",Entry Level,Male,0,0,0
440,Microsoft,218000,"Seattle, WA",Senior,Male,0,0,0
444,ebay,180000,"San Jose, CA",Senior,Female,0,0,0
454,Twitter,500000,"San Francisco, CA",Entry Level,Male,0,0,0
495,Facebook,370000,"Seattle, WA",Junior,Male,0,0,0


In [12]:
def US_location_to_state(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        state = re.sub('(\w+\s)*(\w+),\s','', city)
        
        # return state
        return state
    # otherwise, return NaN
    else:
        return np.nan
    

def location_to_city(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        city = re.sub(',\s([A-Z]{2})$','', city)
        
        # return state
        return city
    # otherwise, return NaN
    else:
        return np.nan
clean_df = clean_df.loc[clean_df.location != "Tel Aviv, Israel"]
clean_df


,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",Entry Level,Male,0,0,0
440,Microsoft,218000,"Seattle, WA",Senior,Male,0,0,0
444,ebay,180000,"San Jose, CA",Senior,Female,0,0,0
454,Twitter,500000,"San Francisco, CA",Entry Level,Male,0,0,0
495,Facebook,370000,"Seattle, WA",Junior,Male,0,0,0
...,...,...,...,...,...,...,...,...
62240,IBM,155000,"Washington, DC",Junior,Male,0,0,0
62283,Facebook,150000,"Singapore, SG, Singapore",Entry Level,Female,0,0,0
62285,ADP,185000,"Roseland, NJ",Junior,Male,0,0,0
62529,Google,685000,"Kirkland, WA",Senior,Male,0,0,0


In [13]:
clean_df['state'] = clean_df.location.str.extract(f'({form_state_one}|{form_state_two})', flags=re.IGNORECASE)[0].apply(US_location_to_state)

In [14]:
clean_df = clean_df[clean_df['state'].notna()]
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state
419,LinkedIn,233000,"San Francisco, CA",Entry Level,Male,0,0,0,CA
440,Microsoft,218000,"Seattle, WA",Senior,Male,0,0,0,WA
444,ebay,180000,"San Jose, CA",Senior,Female,0,0,0,CA
454,Twitter,500000,"San Francisco, CA",Entry Level,Male,0,0,0,CA
495,Facebook,370000,"Seattle, WA",Junior,Male,0,0,0,WA


In [15]:
clean_df['city'] = clean_df.location.str.extract(f'({form_state_one})', flags=re.IGNORECASE)[0].apply(location_to_city)
clean_df.drop('location', axis=1, inplace=True)
clean_df.head()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,company,totalyearlycompensation,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
419,LinkedIn,233000,Entry Level,Male,0,0,0,CA,San Francisco
440,Microsoft,218000,Senior,Male,0,0,0,WA,Seattle
444,ebay,180000,Senior,Female,0,0,0,CA,San Jose
454,Twitter,500000,Entry Level,Male,0,0,0,CA,San Francisco
495,Facebook,370000,Junior,Male,0,0,0,WA,Seattle


In [16]:
clean_df.drop(columns=['gender'], inplace=True)
clean_df.nunique()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


company                    397
totalyearlycompensation    375
yearsofexperience            3
Masters_Degree               2
Bachelors_Degree             2
Doctorate_Degree             2
state                       39
city                       159
dtype: int64

In [17]:
print(clean_df["company"].value_counts().head(10))

Amazon          224
Microsoft       202
Facebook        175
Google           83
Apple            75
Uber             60
IBM              58
Capital One      44
LinkedIn         34
Walmart Labs     33
Name: company, dtype: int64


In [18]:
# #Binning he companies- .cut with 4 bins...do we want to specifically design these?
# bin_labels = ['Small', 'Medium', 'Large']
# bins3 = clean_df['company'].value_counts()
# company_bins = pd.cut(bins3, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
# company_bins

In [19]:
#Binning he companies- .cut with 4 bins...do we want to specifically design these?
bin_labels = ['Small', 'Medium', 'Large', 'Very Large']
bins4 = clean_df['company'].value_counts()
company_bins = pd.cut(bins4, bins=4, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
company_bins


['Very Large',
 'Very Large',
 'Very Large',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 

In [20]:
#change company name to bin name
companies = []
for company in clean_df['company']:
    if company == "Amazon" or company == "Microsoft" or company == "Facebook":
        company = "Large"
    elif company == "Google" or company == "Apple" or company == "IBM" or company=="Uber":
        company = "Medium"
    else:
        company = "Small"
    companies.append(company)

In [21]:
#reassign company column to bin name
clean_bins_df = clean_df.copy()
clean_bins_df['company'] = companies
# clean_bins_df.drop(columns=['Doctorate_Degree'], inplace=True)
# clean_bins_df.drop(columns=['city'], inplace=True)
clean_bins_df.head(10)
# clean_bins_df['gender'].value_counts()

,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
419,Small,233000,Entry Level,0,0,0,CA,San Francisco
440,Large,218000,Senior,0,0,0,WA,Seattle
444,Small,180000,Senior,0,0,0,CA,San Jose
454,Small,500000,Entry Level,0,0,0,CA,San Francisco
495,Large,370000,Junior,0,0,0,WA,Seattle
499,Large,200000,Entry Level,0,0,0,WA,Seattle
509,Large,340000,Senior,0,0,0,WA,Bellevue
510,Medium,690000,Senior,0,0,0,WA,Kirkland
513,Small,600000,Entry Level,0,0,0,CA,Los Gatos
523,Small,168000,Junior,0,0,0,CA,Palo Alto


In [22]:
# #is there linearity between location numerical representation and salary?
# clean_bins2 = clean_bins_df.drop(['location'], axis = 1)
# locations = clean_bins_df['location']
# clean_bins2.head()

In [23]:
# #Create label Encoder and encode categorical
# from sklearn.preprocessing import LabelEncoder
# lb_make = LabelEncoder()
# clean_bins2['company']= lb_make.fit_transform(clean_bins2['company'])
# clean_bins2['gender']= lb_make.fit_transform(clean_bins2['gender'])

# clean_bins2.head()

In [24]:
# clean_bins2['gender'].value_counts() #what do we do with 4 genders?

In [25]:
# #OneHot to new categories
# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(sparse=False)
# company_encoded = lb_make.fit_transform(clean_bins2['company'])
# company_encoded = company_encoded.reshape(len(company_encoded), 1)
# onehot_company_encoded = enc.fit_transform(company_encoded)
# print(onehot_company_encoded)

In [26]:
#skip procedural encoding with get_dummies()?
# clean_dummies = clean_dummies.drop(['gender_2'], axis=1)
clean_dummies = pd.get_dummies(data=clean_bins_df, columns=['company', 'city', 'state', 'yearsofexperience'], drop_first=True)#drop the gender columns?

clean_dummies.head(15)

,totalyearlycompensation,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Medium,company_Small,city_Alexandria,city_Alpharetta,city_Ann Arbor,city_Annapolis Junction,...,state_RI,state_SC,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,yearsofexperience_Junior,yearsofexperience_Senior
419,233000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440,218000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
444,180000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
454,500000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,370000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
499,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
509,340000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
510,690000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
513,600000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
523,168000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
X = clean_dummies.drop(columns=["totalyearlycompensation"])
y = clean_dummies["totalyearlycompensation"]
X.shape
# Scale the dataset using MinMaxScaler()
X_scaled = MinMaxScaler().fit_transform(X)
X_scaled

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                    y, 
                                                    random_state=42)

In [29]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.45599258498410455
Testing Score: -1.9271208541386244e+23


In [30]:
#LinReg basically could not do better than true random... testing random Forest Regressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=42)

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 42)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor(n_estimators=200, random_state=42)

In [32]:
Y_pred = regressor.predict(X_test)  # test the output by changing values


In [33]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.6216690904086696
Testing Score: 0.2824973160334444


In [34]:
# RFR model looks pretty good... 
#could we use cross validation and/or decision trees to determine which features are not contributing to the model
#then tune by removing these noise contributors

In [35]:
#neural network?- go back to clean_bins_df
# Generate our categorical variable lists
app_cats = clean_bins_df.dtypes[clean_bins_df.dtypes == "object"].index.tolist()
clean_bins_df[app_cats].nunique()

company                3
yearsofexperience      3
state                 39
city                 159
dtype: int64

In [36]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(clean_bins_df[app_cats]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cats)
encode_df.head()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,company_Large,company_Medium,company_Small,yearsofexperience_Entry Level,yearsofexperience_Junior,yearsofexperience_Senior,state_AL,state_AR,state_AZ,state_CA,...,city_Walnut Creek,city_Waltham,city_Warren,city_Washington,city_Watertown,city_Wellesley,city_West McLean,city_Wilmington,city_Woonsocket,city_Worcester
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Merge one-hot encoded features and drop the originals
clean_bins_df = clean_bins_df.merge(encode_df, left_index=True, right_index = True)
clean_bins_df.drop(app_cats,1, inplace=True)
clean_bins_df.head()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,totalyearlycompensation,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Large,company_Medium,company_Small,yearsofexperience_Entry Level,yearsofexperience_Junior,yearsofexperience_Senior,...,city_Walnut Creek,city_Waltham,city_Warren,city_Washington,city_Watertown,city_Wellesley,city_West McLean,city_Wilmington,city_Woonsocket,city_Worcester
419,233000,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440,218000,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
444,180000,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
454,500000,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,370000,0,0,0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
clean_bins_df.totalyearlycompensation.values

array([233000, 218000, 180000, 500000, 370000, 200000, 340000, 690000,
       600000, 168000, 187000, 225000, 220000, 130000, 112000, 200000,
       120000, 500000, 220000, 147000, 340000, 235000, 206000, 233000,
       149000, 356000, 305000, 200000, 215000, 700000, 243000, 260000,
       240000, 300000, 160000, 220000, 210000, 210000, 115000, 115000,
       188000, 215000, 315000, 240000, 230000, 216000, 125000, 322000,
       400000, 157000,  82000, 120000, 240000, 100000, 114000, 119000,
       250000, 182000, 120000, 318000, 135000, 228000, 137000, 156000,
       193000, 350000])

In [39]:
# Split our preprocessed data into our features and target arrays
y = clean_bins_df.totalyearlycompensation.values
X = clean_bins_df.drop(columns=['totalyearlycompensation'])
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [40]:
#  Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
# hidden_nodes_layer2 = 20
# hidden_nodes_layer3 = 14

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# #Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 10)                2080      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 11        
Total params: 2,091
Trainable params: 2,091
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [50]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 


Epoch 1/50
2/2 [==============================] - 0s 2ms/step - loss: 1690867.8750 - accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 4ms/step - loss: 1458446.9167 - accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 3ms/step - loss: 1196950.1667 - accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 6ms/step - loss: 1253952.3333 - accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 3ms/step - loss: 1393837.4583 - accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 4ms/step - loss: 1153030.4167 - accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 5ms/step - loss: 1153347.2500 - accuracy: 0.0000e+00
Epoch 8/50
2/2 [==============================] - 0s 7ms/step - loss: 1178658.2917 - accuracy: 0.0000e+00
Epoch 9/50
2/2 [==============================] - 0s 9ms/step - loss: 1094195.5833 - accuracy: 0.0000e+00
Epoch 10/50
2/2 [=============================

In [53]:
#RF?
#Create variables from df
# Split our preprocessed data into our features and target arrays
y = clean_bins_df.totalyearlycompensation.values
X = clean_bins_df.drop(columns=['totalyearlycompensation'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/50
2/2 [==============================] - 0s 3ms/step - loss: 2313622.1667 - accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 5ms/step - loss: 2122169.0833 - accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 8ms/step - loss: 1688093.4583 - accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 8ms/step - loss: 1540266.2500 - accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 10ms/step - loss: 1532843.9167 - accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 7ms/step - loss: 1323684.5000 - accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 11ms/step - loss: 1138040.3958 - accuracy: 0.0000e+00
Epoch 8/50
2/2 [==============================] - 0s 6ms/step - loss: 1121586.2917 - accuracy: 0.0000e+00
Epoch 9/50
2/2 [==============================] - 0s 10ms/step - loss: 1132327.2917 - accuracy: 0.0000e+00
Epoch 10/50
2/2 [==========================